## Always run thic CELL below
- To attach ESML controlplane to your project
- To point at `template-data` for the pipelinbe to know the schema of data
- To init the ESMLPieplinefactory

In [1]:
import sys
sys.path.insert(0, "../azure-enterprise-scale-ml/esml/common/")
from esml import ESMLProject
from baselayer_azure_ml_pipeline import ESMLPipelineFactory, esml_pipeline_types

p = ESMLProject()
p.inference_mode = True
p.active_model = 11 # 10=titanic , 11=Diabetes
p_factory = ESMLPipelineFactory(p)

scoring_date = '1000-01-01 10:35:01.243860' # 
p_factory.batch_pipeline_parameters[1].default_value = scoring_date # overrides ESMLProject.date_scoring_folder.
p_factory.describe()


Using lake_settings.json with ESML version 1.4 - Models array support including LABEL

 ---- Q: WHICH files are generated as templates, for you to EDIT? ---- 
A: These files & locations:
File to EDIT (step: IN_2_SILVER_1): ../../../2_A_aml_pipeline/4_inference/batch/M10/in2silver_ds01_titanic.py
File to EDIT (step: IN_2_SILVER_2): ../../../2_A_aml_pipeline/4_inference/batch/M10/in2silver_ds02_haircolor.py
File to EDIT (step: IN_2_SILVER_3): ../../../2_A_aml_pipeline/4_inference/batch/M10/in2silver_ds03_housing.py
File to EDIT (step: IN_2_SILVER_4): ../../../2_A_aml_pipeline/4_inference/batch/M10/in2silver_ds04_lightsaber.py
File to EDIT (step: SILVER_MERGED_2_GOLD): ../../../2_A_aml_pipeline/4_inference/batch/M10/silver_merged_2_gold.py
File to EDIT (step: SCORING_GOLD): ../../../2_A_aml_pipeline/4_inference/batch/M10/scoring_gold.py
File to EDIT (step: TRAIN_SPLIT_AND_REGISTER): ../../../2_A_aml_pipeline/4_inference/batch/M10/train_split_and_register.py
File to EDIT (step: TRAIN_MANUA

# IN_2_GOLD_SCORING
- Full ML-workflow: If you want to refine data from IN to GOLD, and SCORE model on GOLD, saves SCORED_GOLD in datalake
- Scenario: You want MLOps and full automation, ESMLPipelineFactory starting from Azure Data factory, and calling this genereated Azure ML Pipeline. 
    - Pipeline saving data automatically using the enterprise datalake/ESML AutoLake and ESML SDK

In [2]:
## BUILD
p_factory.create_dataset_scripts_from_template(overwrite_if_exists=True) # Do this once, then edit them manually. overwrite_if_exists=False is DEFAULT
batch_pipeline = p_factory.create_batch_pipeline(esml_pipeline_types.IN_2_GOLD_SCORING)

Creates template step_files.py for user to edit at:
Edit at ../../../2_A_aml_pipeline/4_inference/batch/M10/in2silver_ds01_titanic.py
Edit at ../../../2_A_aml_pipeline/4_inference/batch/M10/in2silver_ds02_haircolor.py
Edit at ../../../2_A_aml_pipeline/4_inference/batch/M10/in2silver_ds03_housing.py
Edit at ../../../2_A_aml_pipeline/4_inference/batch/M10/in2silver_ds04_lightsaber.py
Edit at ../../../2_A_aml_pipeline/4_inference/batch/M10/silver_merged_2_gold.py
Edit at ../../../2_A_aml_pipeline/4_inference/batch/M10/scoring_gold.py
Edit at ../../../2_A_aml_pipeline/4_inference/batch/M10/train_split_and_register.py
Edit at ../../../2_A_aml_pipeline/4_inference/batch/M10/train_manual.py
Edit at ../../../2_A_aml_pipeline/4_inference/batch/M10/your_code/your_custom_code.py
Using GEN2 as Datastore
use_project_sp_2_mount: True
ESML will auto-create a compute...
Note: OVERRIDING enterprise performance settings with project specifics. (to change, set flag in 'dev_test_prod_settings.json' -> ove

In [3]:
## RUN
pipeline_run = p_factory.execute_pipeline(batch_pipeline)
pipeline_run.wait_for_completion(show_output=False)

execute_pipeline (scoring): Inference_mode: 1
Created step IN 2 SILVER - ds01_titanic [eb669812][55925d81-1501-48a7-b414-c662e374b1fb], (This step will run and generate new outputs)
Created step IN 2 SILVER - ds02_haircolor [1ddc6965][48803185-7e76-41c5-b8fc-4e9dd92aab9e], (This step will run and generate new outputs)
Created step IN 2 SILVER - ds03_housing [6f6f2157][61949003-ff39-4c84-87fa-66e86d4140bb], (This step will run and generate new outputs)
Created step IN 2 SILVER - ds04_lightsaber [0fee0835][c77ba037-ca83-447d-891d-c71bdd419f74], (This step will run and generate new outputs)
Created step SILVER MERGED 2 GOLD [e569304b][e963fe5b-1c97-45c1-bcc2-a759b8376d80], (This step will run and generate new outputs)
Created step SCORING GOLD [f4b5eb80][8554d554-6efc-493b-8b0e-82637842eeed], (This step will run and generate new outputs)
Submitted PipelineRun f1efb39a-c578-4062-828b-1037bb7164c0
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/f1efb39a-c578-4062-828b-1037b

'Canceled'

In [4]:
# PUBLISH
published_pipeline, endpoint = p_factory.publish_pipeline(batch_pipeline,"_1") # "_1" is optional    to create a NEW pipeline with 0 history, not ADD version to existing pipe & endpoint

pub_pipe.name 11_diabetes_model_reg_pipe_IN_2_GOLD_SCORING_EP_6
pub_pipe.id 01274209-95e8-4ffb-9b95-9bfb1c865b28
pub_pipe.name 11_diabetes_model_reg_pipe_IN_2_GOLD_SCORING_EP_1
pub_pipe.id cb6e1ef3-ddf2-483c-964e-bc6ba02c610f
pub_pipe.name 10_titanic_model_clas_pipe_IN_2_GOLD_SCORING_EP_1
pub_pipe.id 6c3c51e9-df26-4373-ae37-0a31a3616ec0


# Get info - to use  in Azure data factory: `published_pipeline.id` (if private Azure ML workspace)

In [5]:
print("2) Fetch scored data: Below needed for Azure Data factory PIPELINE activity (Pipeline OR Endpoint. Choose the latter") 
print ("- Endpoint ID")
print("Endpoint ID:  {}".format(endpoint.id))
print("Endpoint Name:  {}".format(endpoint.name))
print("Experiment name:  {}".format(p_factory.experiment_name))

print("In AZURE DATA FACTORY - This is the ID you need, if using PRIVATE LINK, private Azure ML workspace.")
print("-You need PIPELINE id, not pipeline ENDPOINT ID ( since cannot be chosen in Azure data factory if private Azure ML)")
published_pipeline.id

2) Fetch scored data: Below needed for Azure Data factory PIPELINE activity (Pipeline OR Endpoint. Choose the latter
- Endpoint ID
Endpoint ID:  6c3c51e9-df26-4373-ae37-0a31a3616ec0
Endpoint Name:  10_titanic_model_clas_pipe_IN_2_GOLD_SCORING_EP_1
Experiment name:  10_titanic_model_clas_pipe_IN_2_GOLD_SCORING
In AZURE DATA FACTORY - This is the ID you need, if using PRIVATE LINK, private Azure ML workspace.
-You need PIPELINE id, not pipeline ENDPOINT ID ( since cannot be chosen in Azure data factory if private Azure ML)


'0a3ffd65-f1e2-4705-8fd0-681e4d16fbe4'